In [4]:
# Librerias
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

import plotly.graph_objects as go

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense

In [9]:
# Definir ticker y rango de fechas
ticker = "TTWO"
start_date = "2010-01-01"
end_date = None

# Descargar datos
data = yf.download(ticker, start=start_date, end=end_date)

# Aplanar columnas en caso de que vengan con MultiIndex
if isinstance(data.columns, pd.MultiIndex):
    data.columns = [' '.join(col).strip() for col in data.columns.values]

# Verificamos si existe la columna 'Close'
if 'Close' in data.columns:
    close_series = data['Close']
else:
    # En algunos casos se llama 'Close Algo', buscamos la columna que contiene 'Close'
    col_close = [c for c in data.columns if 'Close' in c][0]
    close_series = data[col_close]



fig = go.Figure()
fig.add_trace(go.Scatter(x=close_series.index,
                         y=close_series.values,
                         mode='lines',
                         name='Closing Price'))
fig.update_layout(title="Precio de cierre diario",
                  xaxis_title="Fecha",
                  yaxis_title="Precio de cierre")
fig.show()

C:\Users\canal\AppData\Local\Temp\ipykernel_26640\3015644594.py:7: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [10]:
# Convertir a numpy para el modelo
serie = close_series.values
# Definir tamaño de ventana
window_size = 20

def crear_ventanas(datos, window_size):
    X, y = [], []
    for i in range(len(datos) - window_size):
        X.append(datos[i : i + window_size])
        y.append(datos[i + window_size])
    return np.array(X), np.array(y)

In [11]:
porcentaje_train = 0.8
n_train = int(len(serie) * porcentaje_train)

# Ajustamos tamaño a (n,1) para que el minmax scaler funcione correctamente
serie = serie.reshape(-1, 1)
serie_train = serie[:n_train] # Solo el pasado
serie_test  = serie[n_train:] # Solo el futuro

In [12]:
#  Ajustar el MinMaxScaler SOLO con el entrenamiento
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(serie_train)

# Transformar train y test con el scaler ya entrenado
serie_train_scaled = scaler.transform(serie_train) 
serie_test_scaled  = scaler.transform(serie_test)  

In [13]:
X_train, y_train = crear_ventanas(serie_train_scaled, window_size)
X_test,  y_test  = crear_ventanas(serie_test_scaled,  window_size)

In [14]:
try:
  import treescope
except ImportError:
  !pip install treescope

from __future__ import annotations

import typing
from typing import Any
import dataclasses
import jax
import jax.numpy as jnp
import numpy as np
import torch
import IPython
import treescope

In [15]:
with treescope.active_autovisualizer.set_scoped(treescope.ArrayAutovisualizer()):
  treescope.display([X_train, y_train])

In [16]:

model = Sequential([
    Input(shape=(window_size, 1)),
    LSTM(units=10),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 10)             │           480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 491 (1.92 KB)

 Trainable params: 491 (1.92 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
hist = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

Epoch 1/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.1246 - val_loss: 0.0569
Epoch 2/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0057 - val_loss: 0.0037
Epoch 3/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0017 - val_loss: 0.0030
Epoch 4/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0013 - val_loss: 0.0028
Epoch 5/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0011 - val_loss: 0.0027
Epoch 6/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8.9214e-04 - val_loss: 0.0026
Epoch 7/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7.4727e-04 - val_loss: 0.0029
Epoch 8/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.5233e-04 - val_loss: 0.0025
Epoch 9/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.5978e-04 - val_loss: 0.0025
Epoch 10/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.0585e-04 - val_loss: 0.0021
Epoch 11/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.4302e-04 - val_loss: 0.0017
Epoch 12/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

In [18]:
# Predicciones en test pero resultado esta en escala {0,1}
y_pred_scaled = model.predict(X_test)

# Convertir a columna (N,1)
y_test_scaled_2d = y_test.reshape(-1,1)
y_pred_scaled_2d = y_pred_scaled.reshape(-1,1)

# Inverse transform
y_test_real = scaler.inverse_transform(y_test_scaled_2d).flatten()
y_pred_real = scaler.inverse_transform(y_pred_scaled_2d).flatten()

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


In [19]:
# Ajustar índices de tiempo para alinearlos con la serie original
idx_inicio_test = n_train + window_size
idx_start_test = n_train + window_size
idx_end_test = idx_start_test + len(y_test_real)

fechas_test = close_series.index[idx_start_test : idx_end_test]

In [20]:
# Graficar resultados en escala real

fig = go.Figure()

# Serie completa real
fig.add_trace(go.Scatter(
    x=close_series.index,
    y=close_series.values,
    mode="lines",
    name="Serie real completa"
))

# Valores reales del test
fig.add_trace(go.Scatter(
    x=fechas_test,
    y=y_test_real,
    mode="lines",
    name="Valor real (test)"
))

# Predicciones ya invertidas
fig.add_trace(go.Scatter(
    x=fechas_test,
    y=y_pred_real,
    mode="lines",
    name="Predicción LSTM (test)",
    line=dict(dash="dot")
))

fig.update_layout(
    title="Predicción sobre el conjunto de prueba (escala real)",
    xaxis_title="Fecha",
    yaxis_title="Precio de Acción"
)

fig.show()